In [ ]:

import aslid.model as model
import aslid.data as data
import numpy as np

from sktime.classification.hybrid import HIVECOTEV2
from sktime.classification.kernel_based import RocketClassifier
from sklearn.model_selection import train_test_split

#import matplotlib.pyplot as plt
import aslid.transformer_model as transformer_model

In [ ]:
X, Y = data.load_training_data('data/train.csv', 'data/sign_to_prediction_index_map.json', limit=1000)
X = data.make_uniform(X, model.FRAMES)

In [ ]:
print('N, frames, landmarks, coordinates')
print(X.shape)

In [ ]:
pipeline = model.PreprocessingPipeline(verbose=True)

X = pipeline.fit_transform(X)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.4, random_state=1)


In [ ]:
print("N, features, frames")
print(X.shape)

In [ ]:
batch_size = 10
input_shape=(X.shape[1:])
print(input_shape)
classifier = transformer_model.TransformerNetwork(
    input_shape=input_shape,
    n_classes=250,
    head_size=X.shape[1],
    num_heads=X.shape[1],
    ff_dim=4,
    num_transformer_blocks=2,
    mlp_units=[8],
    mlp_dropout=0.4,
    dropout=0.25,
)

classifier.compile_args(loss="sparse_categorical_crossentropy",)

#classifier = RocketClassifier()
classifier.fit(X_train,Y_train)

In [ ]:
sign = classifier.predict(X_test)
accuracy = sum(sign == Y_test) / len(Y_test)

In [ ]:
accuracy